In [72]:
import pandas as pd
data = pd.read_csv('skupovi_podataka/happiness.csv')
data.head()

,Country,Region,HappinessRank,HappinessScore,StandardError,Economy,Family,Health,Freedom,GovCorruption,Generosity,DystopiaResidual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [73]:
categoric_columns = ['Country', 'Region'] # iako nećemo koristit Country, pretvorit ćemo sve

for column in categoric_columns:
  data[column] = data[column].astype('category').cat.codes

In [74]:
data.head()

,Country,Region,HappinessRank,HappinessScore,StandardError,Economy,Family,Health,Freedom,GovCorruption,Generosity,DystopiaResidual
0,135,9,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,58,9,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,37,9,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,105,9,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,24,5,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [75]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[['Region', 'StandardError', 'Economy', 'Family', 'Health', 'Freedom', 'GovCorruption', 'Generosity', 'DystopiaResidual']], 
                                                    data[['HappinessScore']], test_size=0.25, random_state=2) 

In [76]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test) # Moramo testni skup sklairati na isti način kao i trening skup

In [77]:
# Napravit ćemo podjelu na True i False kao primjer za binarnu klasifikaciju.

y_train_binary = y_train > 4.5
y_train_binary[:5]

y_test_binary = y_test > 4.5
y_test_binary[:5]

,HappinessScore
12,True
3,True
152,False
6,True
120,True


In [78]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=3)
knc.fit(X_train, y_train_binary.values.ravel())
knc.score(X_test, y_test_binary.values.ravel())

0.85

In [79]:
import keras

from keras.models import Sequential # Sekvencijalni model neuronskih mreža
from keras.layers import Dense # Sloj neuronske mreže

In [83]:
# Binarna klasifikacija

classifier = Sequential() 
classifier.add(Dense(units=12, kernel_initializer='normal', activation='relu', input_dim=9))
classifier.add(Dense(units=4, kernel_initializer='normal', activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid'))
classifier.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

classifier.fit(X_train, y_train_binary, batch_size=5, epochs=40)

Epoch 1/40
118/118 [==============================] - 0s 644us/step - loss: 0.2478 - accuracy: 0.6525
Epoch 2/40
118/118 [==============================] - 0s 136us/step - loss: 0.2386 - accuracy: 0.7881
Epoch 3/40
118/118 [==============================] - 0s 127us/step - loss: 0.2283 - accuracy: 0.7881
Epoch 4/40
118/118 [==============================] - 0s 127us/step - loss: 0.2162 - accuracy: 0.7881
Epoch 5/40
118/118 [==============================] - 0s 127us/step - loss: 0.2029 - accuracy: 0.7881
Epoch 6/40
118/118 [==============================] - 0s 119us/step - loss: 0.1905 - accuracy: 0.7881
Epoch 7/40
118/118 [==============================] - 0s 127us/step - loss: 0.1812 - accuracy: 0.7881
Epoch 8/40
118/118 [==============================] - 0s 127us/step - loss: 0.1734 - accuracy: 0.7881
Epoch 9/40
118/118 [==============================] - 0s 127us/step - loss: 0.1689 - accuracy: 0.7881
Epoch 10/40
118/118 [==============================] - 0s 127us/step - loss: 0.164

In [84]:
y_test_binary[:5] # Prvih 5 stvarnih vrijednosti

,HappinessScore
12,True
3,True
152,False
6,True
120,True


In [87]:
classifier.predict(X_test_scaled[:5]) > 0.5 # Previđene vrijednosti

array([[ True],
       [ True],
       [False],
       [ True],
       [False]])

In [92]:
# Regresija

regressor = Sequential() 
regressor.add(Dense(units=12, kernel_initializer='normal', activation='relu', input_dim=9))
regressor.add(Dense(units=4, kernel_initializer='normal', activation='relu'))
regressor.add(Dense(units=1))
regressor.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
regressor.fit(X_train, y_train, batch_size=4, epochs=40)

Epoch 1/40
118/118 [==============================] - 0s 669us/step - loss: 29.5304 - mean_squared_error: 29.5304
Epoch 2/40
118/118 [==============================] - 0s 212us/step - loss: 27.9609 - mean_squared_error: 27.9610
Epoch 3/40
118/118 [==============================] - 0s 186us/step - loss: 24.8991 - mean_squared_error: 24.8991
Epoch 4/40
118/118 [==============================] - 0s 144us/step - loss: 20.0394 - mean_squared_error: 20.0394
Epoch 5/40
118/118 [==============================] - 0s 144us/step - loss: 13.7211 - mean_squared_error: 13.7211
Epoch 6/40
118/118 [==============================] - 0s 144us/step - loss: 7.7426 - mean_squared_error: 7.7426
Epoch 7/40
118/118 [==============================] - 0s 144us/step - loss: 3.9824 - mean_squared_error: 3.9824
Epoch 8/40
118/118 [==============================] - 0s 144us/step - loss: 2.5533 - mean_squared_error: 2.5533
Epoch 9/40
118/118 [==============================] - 0s 144us/step - loss: 2.2148 - mean_squa

In [93]:
y_test[:5] # Stvarne vrijednosti

,HappinessScore
12,7.200
3,7.522
152,3.575
6,7.378
120,4.514


In [94]:
regressor.predict(X_test_scaled[:5]) # Previđene vrijednosti

array([[6.3057423 ],
       [8.180779  ],
       [0.34435347],
       [7.5286345 ],
       [0.3589827 ]], dtype=float32)

In [95]:
regressor.layers

In [96]:
regressor.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 12)                120       
_________________________________________________________________
dense_41 (Dense)             (None, 4)                 52        
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 5         
Total params: 177
Trainable params: 177
Non-trainable params: 0
_________________________________________________________________
